# Worksheet 08

Name: Haotian Shangguan

UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [88]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.353397490644132, 4.292259606811262, 5.152955328667516, 5.089909419572661, 4.880537211208169, 4.591631742727878, 6.158016902665992, 5.5356298438378175, 4.091653733635787, 4.537787615142324]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [89]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[6.065430322881624, 7.716294096997792, 8.481827431580106, 8.503411910827412, 10.341085219674277, 8.509543555980919, 7.351663015190206, 8.2019604416696, 7.642553541862348, 7.71974268042823]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [90]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.71974268042823, 4.537787615142324, 7.642553541862348, 8.2019604416696, 4.091653733635787, 5.5356298438378175, 6.158016902665992, 7.351663015190206, 4.591631742727878, 8.509543555980919]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. Mean
2. Variance
3. probability

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [91]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.537787615142324, 4.091653733635787, 5.5356298438378175, 6.158016902665992, 4.591631742727878]
[7.71974268042823, 7.642553541862348, 8.2019604416696, 7.351663015190206, 8.509543555980919]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.98294396760196,  mean_2 = 7.88509264702626
var_1 = 0.5663891594290286,  var_2 = 0.17221142761697253


from the true value, s1 has mean at 5 and variance at 1. Compare with the result from the kmean, the mean is pretty close between 5 and 4.98 and variance_1 has slightly different than the true value between 0.566 and 1. 

s2 has mean at 8 and variance at 1 as well. Compare with the result, the mean has a huge gap between variance_2 and the true value which is 1. 

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [92]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.71974268042823
probability of observing that point if it came from cluster 0 =  5.9947267113103905e-06
probability of observing that point if it came from cluster 1 =  1.4610323726454855
point =  4.537787615142324
probability of observing that point if it came from cluster 0 =  0.5171991717922071
probability of observing that point if it came from cluster 1 =  2.1170637162413817e-82
point =  7.642553541862348
probability of observing that point if it came from cluster 0 =  1.1474348973372911e-05
probability of observing that point if it came from cluster 1 =  0.8592691195376138
point =  8.2019604416696
probability of observing that point if it came from cluster 0 =  6.818878681025727e-08
probability of observing that point if it came from cluster 1 =  0.42626543439436293
point =  4.091653733635787
probability of observing that point if it came from cluster 0 =  0.20420530264030162
probability of observing that point if it came from cluster 1 =  9.995386462882797e-106
point =

Compare with the true value, the estimated probability is accurate. 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [93]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [ sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.9857141221251196,  mean_2 = 7.88571634793157
var_1 = 0.5722847910965683,  var_2 = 0.17208022194077444


both mean from the estimated value is almost the same as the result from the previous kmeans and variance from the estimate value also same as the previous value. 

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [94]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.71974268042823
probability of observing that point if it came from cluster 0 =  7.713370581934518e-06
probability of observing that point if it came from cluster 1 =  1.456029180849189
point =  4.537787615142324
probability of observing that point if it came from cluster 0 =  0.5131800296179214
probability of observing that point if it came from cluster 1 =  1.4800989981129845e-82
point =  7.642553541862348
probability of observing that point if it came from cluster 0 =  1.455927709475456e-05
probability of observing that point if it came from cluster 1 =  0.8542435025451742
point =  8.2019604416696
probability of observing that point if it came from cluster 0 =  9.656235196458925e-08
probability of observing that point if it came from cluster 1 =  0.42833669999471585
point =  4.091653733635787
probability of observing that point if it came from cluster 0 =  0.20573719620672717
probability of observing that point if it came from cluster 1 =  6.378100075999531e-106
point =  5

The result is slightly more accurate than the previous one. 

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [101]:
assignments = [[] for _ in range(2)]

for p1, p2, d in zip(prob_s0_x, prob_s1_x, data): 
    if p1 > p2: 
        assignments[0].append(d)
    else: 
        assignments[1].append(d)

print("Cluster 0: ", assignments[0])
print("Cluster 1: ", assignments[1])

Cluster 0:  [4.537787615142324, 4.091653733635787, 5.5356298438378175, 6.158016902665992, 4.591631742727878]
Cluster 1:  [7.71974268042823, 7.642553541862348, 8.2019604416696, 7.351663015190206, 8.509543555980919]
